# Handling failed evaluation in parallel execution

In [1]:
from concurrent.futures import ThreadPoolExecutor
import time
import math

from mango.tuner import Tuner
from scipy.stats import uniform
import random

### Objective function

In [2]:
# objfun raise an error when parameter x is <= 0
def objfun(params):
    x = params['x']
    if x <= 0:
        raise ValueError()
    return math.log(x)

### Parallel implementation with error handling

In [3]:
# define the num of parallel workers
n_jobs = 4

# Obj_parallel uses concurrent.futures to parallelize the execution of 
# objfun and handles the failed evaluation
def obj_parallel(params_list):
    futures = []
    params_evaluated = []
    results = []
    
    # here we are use thread executor which is ideal of I/O bound tasks
    # we can also use the ProcessPoolExecutor depending on the use case
    with ThreadPoolExecutor(max_workers=n_jobs) as executor:
        for params in params_list:
            future = (params, executor.submit(objfun, params)) 
            futures.append(future)

        for params, future in futures:
            try:
                result = future.result()
                params_evaluated.append(params)
                results.append(result)
            except ValueError:
                print(f"Value Error raised for {params}")

    return params_evaluated, results


### Calling Mango to optimize objfun

In [4]:

param_dict = {
    'x': uniform(-2, 10),
}

tuner = Tuner(param_dict, obj_parallel, {'batch_size': n_jobs, 'num_iteration': 10})
results = tuner.maximize()

print('best parameters:',results['best_params'])
print('best objective:',results['best_objective'])

Value Error raised for {'x': -1.4909225892493942}
Value Error raised for {'x': -1.6105810066567166}


  0%|          | 0/10 [00:00<?, ?it/s]

Value Error raised for {'x': -0.1556820858981438}
Value Error raised for {'x': -1.17031058194001}
Value Error raised for {'x': -0.43520564628629677}
Value Error raised for {'x': -0.03285622956872114}
Value Error raised for {'x': -1.6884553740468338}
best parameters: {'x': 7.999831206463529}
best objective: 2.079420442265186
